# RSA Cryptografy

When it comes to asymmetric cryptography, RSA (Rivest-Shamir-Adleman) is the default example to explain concepts regarding the topic. The algorithm is one of the first invented (1978) to do so work, rely on a pair of keys, and yhe reversible property of the key to encrypt with one key and roll back to the original message using the other key.

In RSA standard cryptography, two key types are employed, those that will be presented during this document: The RSA public key and the RSA private key and together the private and public key form the key pair needed in the process of encryption and decryption of a message.

The basic principle behind RSA is the multiplication of two factors $p$ and $q$, applying operations on these numbers it is practical to find three very large positive numbers $e$, $d$ and $n$. With these three numbers, it is possible to apply modular exponentiation to reach a strongly cyphered message. To discover the message that originated the cypher text the $p$ and $q$ would be needed, it is possible of factorizing $n$, but $n$ is a large integer, so the process is computationally inviable turning it into a $NP-Complete$ problem.

In [1]:
import base64
import math
import textwrap
import lilRSA

As mentioned $p$ is the first factor, it is a positive integer. $p$ must be a prime number.

$q$ is the second factor, it is a positive integer. $q$ also must be a prime number.

The function `GenPQ` will set the values for $p$ and $q$, for a normal use they must be random very large numbers for two purposes: So it can work with larger messages, and it is going to be harder to find them again factorizing $n$.

In [2]:
def GenPQ(d:int=None)->int:
    p,q = 0,0
    if d == 8:
        p = 71113279
        q = 98327129
    elif d == 16:
        p = 4056555933657341
        q = 6724511755459679
    elif d == 32:
        p = 79307298401562156961148089405447
        q = 30824043223426789498907784588361
    elif d == 64:
        p = 7102262139724624880661914991820000585426491086958693095823804547
        q = 9624173466341420108973789000052858499246519294416603422050306371
    else:
        p = 7102262139724624880661914991820000585426491086958693095823804547
        q = 9624173466341420108973789000052858499246519294416603422050306371
    return p,q

The function `GenPair` generates the key pair. It will use other functions to set the value of the variables needed during the process of performing manipulations with based on RSA standards.

As shown before, we start fining the values for the two factors $p$ and $q$.

In [3]:
def GenPair(d:int=64,p:int=None,q:int=None)->int:
    e,n,phi = 0,0,0
    if p == None and q == None:
        p,q = GenPQ(d)
    e = SetE()
    n = SetN(p,q)
    phi = SetPhi(p,q)
    d = SetD(phi)
    return p,q,e,n,phi,d

Now it is composed the public key, it consists in two components:

- $e$ is the RSA public expoent, a positive integer. As the number is publicly known there is no problem to be reused multiple times, so $e$ usually has the value $65537$. This public exponent should be a prime number and large, not large as the others, but is large. $65537=2^{16}+1$ and the binary representation is $0b1000000000000001$ there are lots of $0$ turning it easy to work with from the perspective of prossessing cicles.

$$
e = 65537
$$

In [4]:
def SetE(e:int=65537)->int:return e

- $n$ the RSA modulus, a positive integer result from the multiplication of $p$ and $q$.

$$
n = p \cdot q
$$

In [5]:
def SetN(p:int, q:int)->int:
    return p * q

$\phi(x)$ represents how many numbers are relatively prime with $x$ so the $gcd$ (greatest common divisor) between $x$ and $y=\{1\leq y < x\}$ must be $1$ for $y$ to be relatively prime with $x$. 

$n$ results from the multiplication of the factors $p$ and $q$, $\phi(n) = \phi(p) \cdot \phi(q)$.  From the fact that $p$ and $q$ are primes, we can suppose that exists $q-1$ numbers relatively prime with $q$, likewise $p$ where $\phi(p)=p-1$:

$$
\phi(n) = (p-1)\cdot (q-1)
$$

In [6]:
def SetPhi(p:int, q:int)->int:
    return (p - 1) * (q - 1)

The function `FindInversePow` will find the **modular multiplicative inverse** of the $e$ ($e^{-1}$) with respect to the modulus $\phi(n)$, known as private key.

The modular multiplicative inverse of $e$ is a big integer (because the modular is big integer) $d$ such that the product $e \cdot d$ is congruent to $1$ inside the class $\phi(n)$. Using the notation, this congruence is written as:
$$
e\cdot d\equiv1\ (mod\ \phi(n))
$$
To find out the inverse of any number when this number is large as those used in cryptography function is not an easy task even for a processor which would use many clock cycles to find the inverse, so to improve this performance theorems are implemented such as extended Euclidean algorithm, Euler's theorem, and so long.

In python we have a build in function called `pow(e,-1,phi(n))` that raises a number to a power exponent and find the modular correspondent to it in class `\phi(n)`.

LilRSA has two ways to find the inverse, by performing a bruteforce to find a $x$ number like $x=(e**n)\%phi$ but it gets very slow as the expoent gets bigger, so by default `pow` is used.

Note that the private key is based on the public key. We need the public key and from it the private  key is calculated. But to discover the private key when we do not know the $\phi(n)$ it ends up being necessary to have the factors. We can calculate from the modulus, but RSA relies on this problem.

I have the goal to create a function that relys on Euclidean algorithm to find the $e^{-1}$.

In [7]:
def SetD(phi:int,e:int=65537)->int:
    return pow(e, -1, phi)

Having the key pair in hands we can write down both keys following the notation:

- Public key: $(n,e)$
- Private key: $(n,d)$

In [8]:
p,q,e,n,phi,d = GenPair(d=16)
print("Public key: (",n,",",e,")")
print("Private key: (",n,",",d,")")

Public key: ( 27278358062558503271571127853539 , 65537 )
Private key: ( 27278358062558503271571127853539 , 25150598683645832258703790006473 )


For a plain text message $m$ the encryptation function is:
$$
c = m^{e}\ mod_n
$$

An exponential operation is applied so we can get back the number in class $n$ it is congruent to, the data needs to be converted from string to integer before calculating the cyphered.

$m$ is a integer between $0$ and $n-1$. If $m$ is not between the specified range, the cyphered message will not be reversed to is original format, $m$ will be bigger than the modulus width and remainder data are lost.

Adding the data obtained from the previous cell:
$$
c = 796...^{65537}\ mod_{272...}
$$

$c$ is ciphertext representative, an integer between $0$ and $n - 1$.

In [9]:
def Encry(m:str,n:int,e:int=65537)->int:
    i = int.from_bytes(m.encode(), byteorder='big')
    return pow(i,e,n)

m = "Hello!"
c = Encry(m,n)
print("Cyphered message:",c)

Cyphered message: 2545276894363010442995951365102


The process of rolling back the cyphered data to the original message is done through the same process but replacing $e$ that represents the public key, to $d$ which represents the private key

Remind that $e \cdot d \equiv 1\ mod_{n}$, it brings the reversible property of modular exponentiation. Raising the cyphered text (that must be represented in its integer format) to the power of $d$ and applying the module $n$ must results on the same integer number that represents the initial message.

$$
c = m^{d}\ mod_n
$$

Replacing variables to the numbers calculated from previus cells:

$$
c = 245...^{251...}\ mod_{272...}
$$

In [10]:
def Decry(c:int,d:int,n:int)->str:
    c = pow(c,d,n)
    leng = math.ceil(c.bit_length() / 8)
    return c.to_bytes(leng, byteorder='big').decode()

m = Decry(c,d,n)
print("Decrypted message:",m)

Decrypted message: Hello!
